# Calculatorium Interactum Dashboard

Welcome to your personal dashboard for tracking your progress through the Calculatorium Interactum course. Here you can view your scores, completed lessons, and reset your progress if needed.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from db_utils import get_progress, reset_progress, get_courses_and_lessons
from ipywidgets import widgets
from IPython.display import display, clear_output, HTML

# Assume user_id 1 for now. In a real app, you'd get this from authentication.
user_id = 1

## Your Progress

In [ ]:
def plot_progress():
    try:
        progress = get_progress(user_id)
        if progress.empty:
            print("No progress data available. Complete some lessons to see your progress!")
            return
        
        # Plot scores
        plt.figure(figsize=(15, 8))
        sns.barplot(x='lesson', y='score', hue='course', data=progress)
        plt.title('Course and Lesson Scores')
        plt.ylim(0, 100)
        plt.xticks(rotation=45, ha='right')
        plt.legend(title='Course', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()

        # Display completion status
        completed = progress['completed'].sum()
        total = len(progress)
        print(f"Completed lessons: {completed}/{total}")

        # Show all completed lessons
        completed_lessons = progress[progress['completed'] == 1]
        if not completed_lessons.empty:
            print("\nCompleted Lessons:")
            for _, lesson in completed_lessons.iterrows():
                print(f"- {lesson['course']}: {lesson['lesson']} (Score: {lesson['score']}%)")
        
        # Calculate and display average score
        avg_score = progress['score'].mean()
        print(f"\nAverage Score: {avg_score:.2f}%")

        # Show progress over time
        if 'timestamp' in progress.columns:
            progress['timestamp'] = pd.to_datetime(progress['timestamp'])
            plt.figure(figsize=(15, 6))
            sns.lineplot(x='timestamp', y='score', hue='course', data=progress)
            plt.title('Progress Over Time')
            plt.xlabel('Date')
            plt.ylabel('Score')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()

        # Show course-specific progress
        display_course_progress(progress)

    except Exception as e:
        print(f"An error occurred while plotting progress: {str(e)}")

def display_course_progress(progress):
    courses = progress['course'].unique()
    print("\nCourse-Specific Progress:")
    for course in courses:
        course_progress = progress[progress['course'] == course]
        total_lessons = len(course_progress)
        completed_lessons = course_progress['completed'].sum()
        avg_score = course_progress['score'].mean()
        
        print(f"\n{course}:")
        print(f"  Completed lessons: {completed_lessons}/{total_lessons}")
        print(f"  Average score: {avg_score:.2f}%")
        
        # Display individual lesson progress
        print("  Lesson Progress:")
        for _, lesson in course_progress.iterrows():
            status = "Completed" if lesson['completed'] else "In Progress"
            print(f"    - {lesson['lesson']}: {status} (Score: {lesson['score']}%)")

        # Visualize course progress
        plt.figure(figsize=(10, 5))
        sns.barplot(x='lesson', y='score', data=course_progress, palette='viridis')
        plt.title(f'{course} - Lesson Scores')
        plt.ylim(0, 100)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

plot_progress()

## Reset Progress

If you want to start over or reset your progress for specific courses or lessons, you can do so here. Be careful, as this action cannot be undone!

In [ ]:
def on_reset_click(b):
    try:
        selected_courses = [course for course, checkbox in course_checkboxes.items() if checkbox.value]
        selected_lessons = [lesson for lesson, checkbox in lesson_checkboxes.items() if checkbox.value]
        
        if selected_courses or selected_lessons:
            reset_progress(user_id, courses=selected_courses, lessons=selected_lessons)
            reset_message = []
            if selected_courses:
                reset_message.append(f"Courses: {', '.join(selected_courses)}")
            if selected_lessons:
                reset_message.append(f"Lessons: {', '.join(selected_lessons)}")
            print(f"Progress reset for {' and '.join(reset_message)}")
        else:
            reset_progress(user_id)
            print("All progress reset.")
        
        clear_output(wait=True)
        plot_progress()
        display_reset_options()
    except Exception as e:
        print(f"An error occurred while resetting progress: {str(e)}")

def display_reset_options():
    global course_checkboxes, lesson_checkboxes
    course_checkboxes = {}
    lesson_checkboxes = {}
    
    try:
        courses_lessons = get_courses_and_lessons(user_id)
        
        if not courses_lessons:
            print("No courses or lessons available. Complete some lessons to see reset options.")
            return
        
        print("Select courses or lessons to reset (leave all unchecked to reset all):")
        
        for course, lessons in courses_lessons.items():
            course_checkbox = widgets.Checkbox(value=False, description=f"Course: {course}")
            course_checkboxes[course] = course_checkbox
            display(course_checkbox)
            
            for lesson in lessons:
                lesson_checkbox = widgets.Checkbox(value=False, description=f"  Lesson: {lesson}")
                lesson_checkboxes[lesson] = lesson_checkbox
                display(lesson_checkbox)
        
        reset_button = widgets.Button(description="Reset Selected Progress")
        reset_button.on_click(on_reset_click)
        display(reset_button)
    except Exception as e:
        print(f"An error occurred while displaying reset options: {str(e)}")

display_reset_options()

## Next Steps

Based on your current progress, here are some suggested next steps:

In [ ]:
def suggest_next_steps():
    try:
        progress = get_progress(user_id)
        if progress.empty:
            print("1. Start with the first lesson in 'Foundations of Pre-Algebra'.")
            return
        
        # Find lessons with scores below 80%
        review_lessons = progress[progress['score'] < 80]
        if not review_lessons.empty:
            print("1. Review these lessons where your score is below 80%:")
            for _, lesson in review_lessons.iterrows():
                print(f"   - {lesson['course']}: {lesson['lesson']} (Score: {lesson['score']}%)")
        
        # Suggest the next incomplete lesson
        incomplete_lessons = progress[progress['completed'] == 0]
        if not incomplete_lessons.empty:
            next_lesson = incomplete_lessons.iloc[0]
            print(f"2. Continue with the next lesson: {next_lesson['course']} - {next_lesson['lesson']}")
        else:
            print("2. Great job! You've completed all available lessons.")
        
        # Calculate overall progress
        overall_progress = (progress['completed'].sum() / len(progress)) * 100
        print(f"\nOverall Course Progress: {overall_progress:.2f}%")
        
        print("\n3. Remember, consistent practice is key to mastering mathematics. Keep up the great work!")

        # Display a motivational message based on progress
        if overall_progress < 25:
            print("\nYou're just getting started! Every step forward is progress.")
        elif overall_progress < 50:
            print("\nYou're making good progress! Keep pushing forward.")
        elif overall_progress < 75:
            print("\nYou're well on your way to mastering these concepts!")
        else:
            print("\nOutstanding work! You're nearly at the finish line!")

    except Exception as e:
        print(f"An error occurred while suggesting next steps: {str(e)}")

suggest_next_steps()